In [3]:
import tensorflow as tf
import cv2
import time
import numpy as np

new_model = tf.keras.models.load_model('best.keras')

# Initialize the camera
cap = cv2.VideoCapture(0) # 0 is the index of the built-in camera, change if you have multiple cameras

# Check if the camera opened successfully
if not cap.isOpened():
    print("Error: Could not open camera.")
    exit()

# Capture frames from the camera
while True:
    ret, frame = cap.read() # read a frame from the camera
    cv2.imwrite('captured_image.jpg', frame)

    time.sleep(5)

    img = tf.keras.utils.load_img(
    'captured_image.jpg', target_size=(160, 160)
    )
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) # Create a batch

    predictions = new_model.predict(img_array)
    #print(predictions)
    score = tf.nn.relu(predictions)
    predictions = tf.where(predictions < 0.5, 0, 1)

    class_names=['No', 'yes']

    message = "This image most likely belongs to {} with a {:.2f} percent confidence.".format(class_names[np.argmax(score)], 100 * np.max(score))
    #print(predictions)
    #print(message)
    if not ret:
        print("Error: Can't receive frame (stream end?). Exiting ...")
        break

    # Display the resulting frame
    cv2.imshow(message, frame)

    # Press 'q' to exit the loop
    if cv2.waitKey(1) == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 13ms/step


In [5]:
new_model = tf.keras.models.load_model('best.keras')
new_model.save('best.h5')

In [6]:
import tensorflow as tf

# Load the Keras model
model = tf.keras.models.load_model('best.h5')

# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the converted model to a .tflite file
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)


INFO:tensorflow:Assets written to: C:\Users\jagme\AppData\Local\Temp\tmp0aurmyui\assets


INFO:tensorflow:Assets written to: C:\Users\jagme\AppData\Local\Temp\tmp0aurmyui\assets


In [17]:
import tensorflow as tf
import cv2
import time
import numpy as np

# Load the TensorFlow Lite model
interpreter = tf.lite.Interpreter(model_path='model.tflite')
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Initialize the camera
cap = cv2.VideoCapture(0) # 0 is the index of the built-in camera, change if you have multiple cameras

# Check if the camera opened successfully
if not cap.isOpened():
    print("Error: Could not open camera.")
    exit()

# Capture frames from the camera
while True:
    ret, frame = cap.read() # read a frame from the camera
    cv2.imwrite('captured_image.jpg', frame)

    time.sleep(.5)

    img = tf.keras.utils.load_img(
    'captured_image.jpg', target_size=(160, 160)
    )
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) # Create a batch

    # Set the tensor to point to the input data to be inferred
    interpreter.set_tensor(input_details[0]['index'], img_array)

    # Run the inference
    interpreter.invoke()

    # Get the results
    predictions = interpreter.get_tensor(output_details[0]['index'])

    score = tf.nn.relu(predictions)
    predictions = np.where(predictions < 0.5, 0, 1)

    class_names=['No', 'Yes']

    message = "{} with {:.2f} ".format(class_names[np.argmax(score)], 100 * np.max(score))

    if not ret:
        print("Error: Can't receive frame (stream end?). Exiting ...")
        break

    cv2.putText(frame, message, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    # Display the resulting frame
    cv2.imshow(message, frame)
    print(message)

    # Press 'q' to exit the loop
    if cv2.waitKey(1) == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()


Yes with 232.75 
No with 218.36 
No with 220.59 
No with 203.46 
Yes with 188.86 
No with 415.58 
Yes with 708.03 
Yes with 631.68 
Yes with 594.00 
Yes with 246.66 
No with 249.97 
No with 279.73 
No with 279.02 
Yes with 589.71 
Yes with 623.89 
Yes with 601.39 
Yes with 307.05 
Yes with 297.56 
Yes with 229.99 
Yes with 222.37 
Yes with 294.33 
Yes with 256.46 
Yes with 211.99 
Yes with 231.45 
Yes with 292.17 
Yes with 620.23 
Yes with 559.17 
Yes with 583.28 
Yes with 617.84 
No with 358.90 
No with 349.21 
No with 356.80 
No with 381.85 
No with 344.69 
Yes with 454.78 
Yes with 759.79 
Yes with 726.39 
No with 565.50 
No with 252.81 
No with 277.64 
No with 557.18 
Yes with 578.77 
Yes with 688.90 
Yes with 660.42 
No with 404.23 
No with 413.65 
No with 402.91 
No with 374.30 


KeyboardInterrupt: 